# Strategy Report Notebook
This notebok runs analysis on a given strategy.

## Import Libraries

In [27]:
import plotly.express as px
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from ipywidgets import interact, interact_manual
import pandas as pd
import plotly.graph_objects as go


## Import Daily, Weekly, Monthly, Quarterly and Yearly Returns Data for Equity Benchmark

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**):  SPTR, M1WD, SX5T

In [33]:
ls_var_m = pd.read_excel(dm.NEW_DATA+'lsvar_1.xlsx', sheet_name='monthly', index_col=0)
ls_var_w = pd.read_excel(dm.NEW_DATA+'lsvar_1.xlsx', sheet_name='weekly', index_col=0)
jpm = pd.read_excel(dm.NEW_DATA+'lsvar_1.xlsx', sheet_name='jpm', index_col=0)
jpm_dict = dm.get_data_dict(jpm, data_type='index')
ls_var_m.columns

Index(['SPTR', 'SG Var', 'SG Var w/ Trend', 'CS L/S Var',
       'CS L/S Var (Short leg rolled daily, after Sep-16)',
       'CS L/S Var (Short leg DH EOD)', 'CS L/S Var (3M VIX Futures Long Leg)',
       'UBS L/S Var 85-15', 'UBS L/S Var 80-20', 'UBS L/S Var 75-25',
       'UBS L/S Var 70-30'],
      dtype='object')

In [34]:
jpm_dict['Weekly'].drop(['Short 1w Variance with Intraday DH'], axis=1, inplace=True)
jpm_dict['Monthly'].drop(['Short 1w Variance with Intraday DH'], axis=1, inplace=True)

In [35]:
ls_var_m.drop(['SG Var', 'SG Var w/ Trend'], axis = 1, inplace = True)
ls_var_w.drop(['SG Var', 'SG Var w/ Trend'], axis = 1, inplace = True)
ret_dict_1 = {'Weekly':ls_var_w, 'Monthly': ls_var_m}

ls_var_m_2 = dm.merge_data_frames(ls_var_m, jpm_dict['Monthly'])
ls_var_w_2 = dm.merge_data_frames(ls_var_w, jpm_dict['Weekly'])
ret_dict_2 = {'Weekly':ls_var_w_2, 'Monthly': ls_var_m_2}


In [36]:
ls_var_m_2.columns

Index(['SPTR', 'CS L/S Var',
       'CS L/S Var (Short leg rolled daily, after Sep-16)',
       'CS L/S Var (Short leg DH EOD)', 'CS L/S Var (3M VIX Futures Long Leg)',
       'UBS L/S Var 85-15', 'UBS L/S Var 80-20', 'UBS L/S Var 75-25',
       'UBS L/S Var 70-30', 'Relative Variance Strategy'],
      dtype='object')

In [ ]:
equity_bmk = 'SPTR'
strat_drop_list = ['99%/90% Put Spread','Down Var', 'VOLA', 'Dynamic Put Spread', 'VRR',
       'GW Dispersion', 'Corr Hedge', 'Def Var']
include_fi = False
#create returns data dictionary for equity benchmark
vortex_dict= dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

## Import Daily, Weekly, Monthly, Quarterly and Yearly Returns Data for a Strategy
To import the returns data:
* In **new_data = get_new_strategy_returns_data( report_name= , sheet_name = )**
    * Add File Name to **report_name** 
    * Add sheet name to **sheet_name**

In [ ]:
new_3_robust= pd.read_excel(dm.NEW_DATA+'Barclays_new3_robustness.xlsx',
                                           sheet_name = 'data', index_col=0)

#create returns data dictionary for strategy
new_data_dict = dm.get_data_dict(new_3_robust)

#merge dictionaries
ret_data_dict = dm.merge_dicts(vortex_dict,new_data_dict)

## View Returns Data
Select frequency (Daily, Weekly, Monthly, Yearly) to view data

In [10]:
freq_list = ['Weekly', 'Monthly']
@interact
def display_returns(frequency = freq_list, ret_dict=[ret_dict_1, ret_dict_2]):
    return fmt.get_returns_styler(ret_dict[frequency])

interactive(children=(Dropdown(description='frequency', options=('Weekly', 'Monthly'), value='Weekly'), Dropdo…

TraitError: Invalid selection: value not found

TraitError: Invalid selection: value not found

TraitError: Invalid selection: value not found

## Compute Return Stats and Hedge Metrics

In [12]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict = summary.get_analytics_data(ret_dict_1)

In [15]:
analytics_dict_1 = summary.get_analytics_data(ret_dict_2)

In [ ]:
col_dict = {'max_vix':['SPTR', 'Vortex', 'new3', 'maxVIXDelta=0.65', 'maxVIXDelta=0.7','maxVIXDelta=0.75', 
                       'maxVIXDelta=0.8', 'maxVIXDelta=0.85','maxVIXDelta=0.9', 'maxVIXDelta=1.1'],
           'short_put_strike': ['SPTR', 'Vortex', 'new3','shortPutStrikes=-0.35,-0.3,-0.25', 
                                'shortPutStrikes=-0.3,-0.25,-0.2','shortPutStrikes=-0.2', 'shortPutStrikes=-0.25',
                                'shortPutStrikes=-0.3','shortPutStrikes=-0.35', 'shortPutStrikes=-0.4'],
           'long_put_strike':['SPTR', 'Vortex', 'new3', 'longSPXStrike=-0.1','longSPXStrike=-0.15'],
           'vix_strike':['SPTR', 'Vortex', 'new3', 'VIXStrike=0.2', 'VIXStrike=0.3'],
           'roll_dates':['SPTR', 'Vortex', 'new3', 'rollDates=-20,-15,-10,-5', 'rollDates=5,10,15,20']}


In [ ]:
for key in col_dict:
    for freq in analytics_dict:
        temp_dict = {}
        temp_rs = analytics_dict[freq][False]['return_stats'].copy()
        temp_rs = temp_rs[col_dict[key]]
        temp_dict['return_stats'] = temp_rs
        temp_hm = analytics_dict[freq][False]['hedge_metrics'].copy()
        temp_hm = temp_hm[col_dict[key][1:]]
        temp_dict['hedge_metrics'] = temp_hm
        analytics_dict[freq][key] = temp_dict

## Display Return Stats and Hedge Metrics
Display Returns Analytics data by:
* **frequency (Monthly or Weekly)** - Show Monthly or Weekly returns
* **stats (return_stats, hedge_metrics)** - Show:
    * Return Statistics or 
    * Hedge Metrics

In [14]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

interactive(children=(Dropdown(description='frequency', options=('Weekly', 'Monthly'), value='Weekly'), Dropdo…

In [16]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict_1[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

interactive(children=(Dropdown(description='frequency', options=('Weekly', 'Monthly'), value='Weekly'), Dropdo…

## Compute Historical Sell Offs

In [ ]:
hist_dict = summary.get_hist_data(ret_data_dict)

In [ ]:
hist_list = list(hist_dict[False].columns)[0:2]
for key in col_dict:
    temp_list = hist_list + col_dict[key]
    temp_hs = hist_dict[False].copy()
    temp_hs = temp_hs[temp_list]
    hist_dict[key] = temp_hs

## Display Historical Sell Offs

In [ ]:
@interact
def display_selloffs(param = [False]+list(col_dict.keys())):
    df_hist = hist_dict[param]
    return fmt.get_hist_styler(df_hist)

In [ ]:
roll_dates = pd.read_excel(dm.NEW_DATA+'Barclays_roll_plus_vix.xlsx',
                                           sheet_name = 'data', index_col=0)

#create returns data dictionary for strategy
new_data_dict_2 = dm.get_data_dict(roll_dates)

#merge dictionaries
ret_data_dict_2 = dm.merge_dicts(vortex_dict, new_data_dict_2)

In [ ]:
analytics_dict_2 = summary.get_analytics_data(ret_data_dict_2)

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict_2[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

In [ ]:
hist_dict_2 = summary.get_hist_data(ret_data_dict_2)

In [ ]:
@interact
def display_selloffs():
    df_hist = hist_dict_2[False]
    return fmt.get_hist_styler(df_hist)

## Compute Quintile or Decile Analysis
Display grouped data by:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles

In [18]:
group_list=['Quintile','Decile']
@interact
def display_returns(group = group_list):
    quintile_df = summary.get_grouped_data(ret_dict_2, group = group)
    return(quintile_df.style.format("{:.2%}"))

interactive(children=(Dropdown(description='group', options=('Quintile', 'Decile'), value='Quintile'), Output(…

## Display Quintile or Decile Analysis
Display Quintile Analysis Bar Chart:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles
* **strat** - Show quintile or decile for selected strategy

In [23]:
group_list=['Quintile','Decile']
@interact

def display_quintile_analysis(group = group_list, strat=list(ret_dict_2['Monthly'].select_dtypes('float').columns)[1:]):    
    df = summary.get_grouped_data(ret_dict_2, group = group)
    
    for col in df.columns:
        if not (col == 'SPTR' or col ==strat):
            df.drop([col], axis=1, inplace=True)
    if group == "Quintile":
        title ='Quintile Analysis'
    else:
        title = 'Decile Analysis'
        
    fig = px.bar(df,barmode='group',title = title)
    fig.show()
    

interactive(children=(Dropdown(description='group', options=('Quintile', 'Decile'), value='Quintile'), Dropdow…

# Create Report

Run this code below to export analysis into excel spreadsheet
* **strat_report (string)**: provide a name for the excel file
* **selloffs (boolean)**:
    * **True**: if you want historical selloff data in the spreadsheet
    * **False**: if you do not want historical selloff data in the spreadsheet

In [37]:
strat_report_1 = 'ls_var_base'
strat_report_2 = 'ls_var_jpm'
selloffs = False
rp.generate_strat_report(strat_report_1, ret_dict_1, selloffs)
rp.generate_strat_report(strat_report_2, ret_dict_2, selloffs)

Computing Monthly Analytics...
Computing Weekly Analytics...
Computing Normalized Hedge Metrics...
"ls_var_base.xlsx" report generated in "C:\Users\NVG9HXP\Documents\Projects\RMP\EquityHedging\reports\" folder
Computing Monthly Analytics...
Computing Weekly Analytics...
Computing Normalized Hedge Metrics...
"ls_var_jpm.xlsx" report generated in "C:\Users\NVG9HXP\Documents\Projects\RMP\EquityHedging\reports\" folder
